In [1]:
# Check the requirements file to ensure all necessary libraries are installed

import sys
sys.path.append("./helper")
from helper import install_requirements

# find the requirements.txt file in the main folder
install_requirements('./requirements.txt')

🚀 Installing packages from requirements.txt...
✅ All packages installed successfully!
⚠️  Dependency conflicts detected:
spyder 5.2.2 requires pyqtwebengine, which is not installed.
sparkmagic 0.21.0 has requirement pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3.
sphinx 8.1.3 has requirement docutils<0.22,>=0.20, but you have docutils 0.19.
spyder 5.2.2 has requirement pyqt5<5.13, but you have pyqt5 5.15.11.



---

## 🧭 Model Configuration Panel
Please use the configuration form below to define how you'd like your hierarchical classification model to behave. This panel allows you to control every major component of the pipeline, including:

🧾 **Dataset Selection**
- Choose Dataset: Select which dataset you'd like to use (dataset1, or dataset2) - 'dataset2 is still under development'
- Label Columns: Enter the column names that represent the hierarchical labels for your selected dataset (e.g., Cat1, Cat2, Cat3).

📝 **Text Feature Selection**

Text Column - Choose which column(s) to use for feature extraction:
- Title: Use only the title column.
- Text: Use only the text/content column.
- Both: Concatenate Title and Text for richer input.

✨ **TF-IDF Preprocessing Settings**

Configure the text-to-numeric transformation:

- Minimum / Maximum Document Frequency: Control rare or overly common term filtering.
- N-gram Range: Specify the size of text chunks to analyze (e.g., unigrams or bigrams).
- Stop Words: Choose a stop-word list (e.g., English) or disable it.

---

🧠 **HiClass Strategies**

HiClass provides multiple strategies for building hierarchical classifiers. Each strategy controls how base classifiers are positioned relative to the hierarchy structure:

🔹 `lcppn` — Local Classifier Per Parent Node (default)
- Trains a separate classifier at **each parent node** to distinguish between its direct children.
- **Advantages**: Efficient and focused. Classifiers learn from local child distributions only.
- **Recommended for**: Large or deep hierarchies where different parent nodes have distinct structures.

🔹 `lcpn` — Local Classifier Per Node
- Trains a classifier at **every node**, including leaf nodes (Can result in many small classifiers, increasing complexity).
- **Advantages**: Fine-grained control and potentially better local precision.
- **Recommended for**: Irregular hierarchies or custom tree control.

🔹 `lcpl` — Local Classifier Per Level
- Trains one classifier per **hierarchical level** (e.g., one for top-level, one for mid-level, etc.).
- **Advantages**: Simple to train. Less memory intensive. Easy to interpret.
- **Recommended for**: Shallow hierarchies or where level-wise accuracy is prioritized.

---

🧠 **Classifier Settings**

Choose which base classifier(s) you'd like to use (**you have an option to choose all three in a once**):

- SGD: Linear model with SVM-style or logistic loss
- LogisticRegression: Classical log-linear model
- RandomForest: Ensemble-based non-linear classifier

Each classifier section includes parameters like:

- Number of estimators, maximum iterations, maximum depth, and criterion for Random Forest
- Max iterations, Loss function (hinge/log) and SVM class weight for SGD

🧪 **Model Calibration & Probabilities**

Calibration/Probability Control:
- none: No probability output or calibration
- calibration only: Apply isotonic/beta/ivap on raw model scores
- probability only: Enable probability combination (e.g., geometric)
- both: Use both calibrated scores and probability combination

- Calibration Method: Choose from isotonic, beta, ivap, or cvap.
- Probability Combiner: Choose how probabilities are merged across hierarchy levels (e.g., geometric, arithmetic, or multiply).

---

In [2]:
# Adding the relevant folders to the python path
sys.path.append("./modules")

from display_utils import build_model_configuration_widgets
from IPython.display import display, HTML

display(HTML("<h2>🛠️ Model Configuration Inputs</h2><p>Please fill in the configuration below to build and train your HiClass model.</p>"))

dynamic_values = build_model_configuration_widgets()

Matplotlib is building the font cache; this may take a moment.


## Model Building
Once you’ve configured your model using above panel, click Run to execute this cell for:

- Train selected models

- Evaluate performance

- View confusion matrices

- Save trained pipelines

In [3]:
# Executing the models based on the given input

from run_workflows import (
    run_logreg_workflow_from_widgets,
    run_rf_workflow_from_widgets,
    run_sgd_workflow_from_widgets 
)

# Dictionary to collect trained models
trained_models = {}

# Iterate over selected classifiers from widgets
for classifier_name in dynamic_values['base_classifiers']:
    clf = classifier_name.lower()

    if clf == 'logisticregression':
        print("⚙️ Running Logistic Regression workflow...")
        model = run_logreg_workflow_from_widgets(dynamic_values)
        trained_models['LogisticRegression'] = model

    elif clf == 'randomforest':
        print("🌲 Running Random Forest workflow...")
        model = run_rf_workflow_from_widgets(dynamic_values)
        trained_models['RandomForest'] = model

    elif clf == 'sgd':
        print("⚙️ Running SGD workflow...")
        model = run_sgd_workflow_from_widgets(dynamic_values)
        trained_models['SGD'] = model

    else:
        print(f"❓ Unknown classifier selected: {classifier_name}")


⚙️ Running SGD workflow...

📊 Evaluation Results for SGD (hinge) with Calibration (None) and Probability (None):
hierarchical_precision: 0.9022035522146068
hierarchical_recall: 0.9022035522146068
hierarchical_f1: 0.9022035522146068
level_accuracy: {'Cat1': 0.9400840150342693, 'Cat2': 0.8965288525315056, 'Cat3': 0.8699977890780456}
exact_path_accuracy: 0.8625912005306212
✅ Model and metadata saved to /home/ec2-user/SageMaker/minds_coder_capability_hiclass/outputs/sgd_model_20251118_045846.joblib
🧩 Confusion Matrices (per level):
📘 Level: Cat1


,baby products,beauty,grocery gourmet food,health personal care,pet supplies,toys games
baby products,577,5,3,7,2,8
beauty,2,1916,6,73,2,15
grocery gourmet food,0,2,532,14,3,4
health personal care,23,119,41,2648,18,33
pet supplies,14,18,6,34,1472,15
toys games,10,11,13,26,15,1359


--------------------------------------------------
📘 Level: Cat2


,action toy figures,arts crafts,baby food,baby toddler toys,bath body,bathing skin care,beverages,birds,breads bakery,breakfast foods,...,sexual wellness,skin care,small animals,snack food,sports outdoor play,strollers,stuffed animals plush,tools accessories,tricycles,vehicles remote control
action toy figures,87,0,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,2
arts crafts,1,69,0,0,2,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0
baby food,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby toddler toys,0,0,0,100,0,1,0,0,0,0,...,0,0,0,0,0,0,6,0,1,1
bath body,0,0,0,0,106,0,1,0,0,0,...,0,6,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
strollers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,20,0,0,0,0
stuffed animals plush,0,0,0,2,0,0,1,0,0,0,...,0,1,0,0,0,0,140,0,1,0
tools accessories,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,56,0,0
tricycles,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,0


--------------------------------------------------
📘 Level: Cat3


...

--------------------------------------------------
📘 Level: Cat2


,action toy figures,arts crafts,baby food,baby toddler toys,bath body,bathing skin care,beverages,birds,breads bakery,breakfast foods,...,sexual wellness,skin care,small animals,snack food,sports outdoor play,strollers,stuffed animals plush,tools accessories,tricycles,vehicles remote control
action toy figures,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arts crafts,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby food,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby toddler toys,0,0,0,16,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bath body,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
strollers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
stuffed animals plush,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
tools accessories,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tricycles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,0


--------------------------------------------------
📘 Level: Cat3


,accessories,activity centers entertainers,activity play centers,adult toys games,air fresheners,albums,allergy,alternative medicine,animals figures,apparel accessories,...,vehicle playsets,vitamins supplements,walkers,water,water treatments,weight loss products,wind up toys,wipes holders,women s,women s health
accessories,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
activity centers entertainers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
activity play centers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adult toys games,0,0,0,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
air fresheners,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weight loss products,0,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,6,0,0,0,0
wind up toys,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wipes holders,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
women s,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,106,0


--------------------------------------------------
📁 Confusion matrices saved to: /home/ec2-user/SageMaker/minds_coder_capability_hiclass/outputs/logistic_reg_confusion_20251118_050029.xlsx


In [4]:
import pandas as pd

def evaluate_and_compare_models(trained_models, X_test_text, y_test, output_path="outputs/model_comparison.xlsx"):
    """
    Evaluates all trained models, compares their metrics, and saves results to Excel.

    Parameters:
    - trained_models: dict of {model_name: model_instance}
    - X_test_text: text input for prediction
    - y_test: true hierarchical labels
    - output_path: where to save the comparison Excel file

    Returns:
    - summary_df: DataFrame of all model metrics
    """
    all_metrics = []

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for model_name, model in trained_models.items():
            print(f"🔍 Evaluating {model_name}...")
            model.predict(X_test_text)
            metrics = model.evaluate(y_test)

            flat_metrics = {
                'model': model_name,
                'hierarchical_f1': metrics['hierarchical_f1'],
                'hierarchical_precision': metrics['hierarchical_precision'],
                'hierarchical_recall': metrics['hierarchical_recall'],
                'exact_path_accuracy': metrics['exact_path_accuracy']
            }

            # Include level-wise accuracy
            for level, acc in metrics['level_accuracy'].items():
                flat_metrics[f'accuracy_{level}'] = acc

            all_metrics.append(flat_metrics)

            # Save full metric details as a sheet
            pd.DataFrame([flat_metrics]).to_excel(writer, sheet_name=model_name, index=False)

    summary_df = pd.DataFrame(all_metrics)
    summary_df.to_excel(output_path, sheet_name="Summary", index=False)

    print(f"✅ Evaluation comparison saved to: {output_path}")
    return summary_df

# This assumes all models were trained and test data is ready
from helper import prepare_text_and_labels, build_dataset_sources_and_labels, build_dataset_map

# Step 1: Get test data
source_dict, label_dict = build_dataset_sources_and_labels(dynamic_values)
dataset_map = build_dataset_map(source_dict, label_dict)
_, X_test_text, _, y_test = prepare_text_and_labels(dynamic_values, dataset_map)

# Step 2: Evaluate all
results_df = evaluate_and_compare_models(trained_models, X_test_text, y_test)
display(results_df)


🔍 Evaluating SGD...
🔍 Evaluating LogisticRegression...
🔍 Evaluating RandomForest...
✅ Evaluation comparison saved to: outputs/model_comparison.xlsx


,model,hierarchical_f1,hierarchical_precision,hierarchical_recall,exact_path_accuracy,accuracy_Cat1,accuracy_Cat2,accuracy_Cat3
0,SGD,0.902204,0.902204,0.902204,0.862591,0.940084,0.896529,0.869998
1,LogisticRegression,0.874788,0.874788,0.874788,0.814614,0.935331,0.867455,0.821579
2,RandomForest,0.264721,0.264721,0.264721,0.150122,0.452355,0.191355,0.150453
